In [11]:
import tweepy
import re
import numpy as np 
import pandas as pd 
import nltk 
nltk.download('stopwords')  
from nltk.corpus import stopwords 
from tweepy import OAuthHandler

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nikhi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


We need to store the Consumer API key, Consumer API secret and Access token and token secret in corresponding string valirables.

In [2]:
consumer_key = 'XXXXXXXXXXX'
consumer_secret = 'XXXXXXXXXXXX'
access_token = 'XXXXXXXXXXXXXXXXXXX'
access_token_secret = 'XXXXXXXXXXXXXXXXXXs' 

The OAuthHandler takes the Consumer API ke and Customer API Secret as argument.Customer API and Secret key tell our client application which application to connect with, While the Acess token and secret key defines the rights to access the application.

In [6]:
authorizer = OAuthHandler(consumer_key, consumer_secret)
authorizer.set_access_token(access_token,access_token_secret)

# Scrapping Tweets
We have successfully connected to the twitter API. Now we have to fetch the tweets.

In [8]:
api = tweepy.API(authorizer,timeout=15)
all_tweets= []
search_query= 'narendra modi'
for tweet_object in tweepy.Cursor(api.search,q=search_query+"-filter:retweets",lang = 'en', result_type='recent').items(500):
    all_tweets.append(tweet_object.text)

We specified that if no tweet are found after seacrhing for 15 seconds then the application should time out. 
all_tweets contain the scraped tweets.
Search_query contains the string which we are looking for in the tweets.
In the loop that uses tweepy's cursor object to fetch tweets.
1. The first parameter is used to tell us what we are performing(We are seraching the tweets)
2. Second parameter is search query where we re provide additional details to it. in our query we are telling it to not to fetch any information form retweets.
3. Third parameter is used to specify what kinf of result type we are looking for.
4. item attributes sets the number of tweets to return.

# Performing Sentimental Analysis
>Data Processing
 To train our model we will be using aline data base from Github. We will divide the data into label and feature set and then will remove special charcter and empty spaces from the tweets.


In [40]:
tweets =pd.read_csv("https://raw.githubusercontent.com/kolaveridi/kaggle-Twitter-US-Airline-Sentiment-/master/Tweets.csv")

X = tweets.iloc[:, 10].values  
y = tweets.iloc[:, 1].values
 
processed_tweets = []
 
for tweet in range(0, len(X)):  
    # Remove all the special characters
    processed_tweet = re.sub(r'\W', ' ', str(X[tweet]))
 
    # remove all single characters
    processed_tweet = re.sub(r'\s+[a-zA-Z]\s+', ' ', processed_tweet)
 
    # Remove single characters from the start
    processed_tweet = re.sub(r'\^[a-zA-Z]\s+', ' ', processed_tweet) 
 
    # Substituting multiple spaces with single space
    processed_tweet= re.sub(r'\s+', ' ', processed_tweet, flags=re.I)
 
    # Removing prefixed 'b'
    processed_tweet = re.sub(r'^b\s+', '', processed_tweet)
 
    # Converting to Lowercase
    processed_tweet = processed_tweet.lower()
 
    processed_tweets.append(processed_tweet)

# TF-IDF for text to Numeric Conversion

TFIDF scheme converts text to numbers. 

In [41]:
from sklearn.feature_extraction.text import TfidfVectorizer  
tfidfconverter = TfidfVectorizer(max_features=2000, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))  
X = tfidfconverter.fit_transform(processed_tweets).toarray()
 

# Training the Sentimental Analysis Model
We will train our model to predict the tweets which we have extrated above. 

In [50]:
from sklearn.ensemble import RandomForestClassifier
text_classifier = RandomForestClassifier(n_estimators=100, random_state=0)  
text_classifier.fit(X, y)


# Predicting Sentiment for Scraped Tweets:
We will be running above algorithm on the scraped tweets which we have generated

In [78]:
p = 0
ni = 0
n = 0
for tweet in all_tweets:
    processed_tweet = re.sub(r'\W', ' ', tweet)
 
    # remove all single characters
    processed_tweet = re.sub(r'\s+[a-zA-Z]\s+', ' ', processed_tweet)
 
    # Remove single characters from the start
    processed_tweet = re.sub(r'\^[a-zA-Z]\s+', ' ', processed_tweet) 
 
    # Substituting multiple spaces with single space
    processed_tweet= re.sub(r'\s+', ' ', processed_tweet, flags=re.I)
 
    # Removing prefixed 'b'
    processed_tweet = re.sub(r'^b\s+', '', processed_tweet)
 
    # Converting to Lowercase
    processed_tweet = processed_tweet.lower()
 
    sentiment = text_classifier.predict(tfidfconverter.transform([ processed_tweet]).toarray())
    if sentiment == ['positive']:
        p = p +1
    elif sentiment == ['negative']:
        ni = ni +1
    else :
        n = n + 1
   # print(processed_tweet ,":", sentiment)
# print(len(all_tweets))
# print( p,ni,n)
if p>ni and p > n :
    print("People on",search_query,"have positive opinion ",(p/(p+n+ni)*100),"%")
elif ni >p and ni > n:
    print("People on",search_query,"have negative opinion of",(ni/(p+n+ni)*100),"%")
elif n> p and n> ni :
    print("People on",search_query,"have neutral opinion of",(n/(p+n+ni)*100),"%")


500
39 120 341
People on narendra modi have neutral opinion of 68.2 %
